In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
h5py.enable_ipython_completer()
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# Script parameters
file_parameters='halo.yml'
displayParameters=True

# Relation expected for isothermal sphere for LCDM cosmology used in Sussing
Delta=100
# Relation expected for isothermal sphere for LCDM cosmology used in Millennium
Delta=200

# Read parameters from yaml file
parameters=yaml.load(open(file_parameters),Loader=yaml.FullLoader)
if displayParameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

haloFile=parameters['inputFiles']['haloFile']
save_prefix=parameters['save_prefix']
graphMin=parameters['graphs']['graphMin']
graphMax=parameters['graphs']['graphMax']
snapMin=parameters['snapshots']['snapMin']
snapMax=parameters['snapshots']['snapMax']
baryonFraction=parameters['cosmo']['baryonFraction']

In [ ]:
# Read in halo table
with h5py.File(haloFile,'r') as f:
    
    #Get the data from the snapshot
    halos=f['Halos'][:]

In [ ]:
# Filter out the halos in our selection.
# Might be faster to include np.nonzero() on rhs, but this packages indices in a tuple
halos=halos[(graphMin<=halos['graph_ID']) & (halos['graph_ID']<=graphMax) & (snapMin<=halos['snap_ID']) & (halos['snap_ID']<=snapMax)]

In [ ]:
for plot in parameters['plots'].items():
    print(plot)
    if plot[1]:
        plotName=plot[0]
        plotProps=parameters[plotName]
        print(plotProps)
        plt.figure(figsize=plotProps['figsize'])
        plt.xscale(plotProps['xscale'])
        plt.yscale(plotProps['yscale'])
        plt.title(plotName)
        plt.xlabel(plotProps['xlabel'])
        plt.ylabel(plotProps['ylabel'])
        if plotName=='velocityDispersion':
            xdata=halos['mass']
            ydata=halos['half_mass_virial_speed']/np.sqrt(2.)
            ydata2=halos['rms_speed']/np.sqrt(3.)
            from astropy.cosmology import FlatLambdaCDM
            cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)
            ydata3=((10*c.M_sun*c.G*cosmo.H(0))**(1/3)).to(u.km/u.s)*(Delta/200)**(1/6)*xdata**(1/3)/np.sqrt(2.)
            plt.plot(xdata,ydata3,'-',label='SIS model, $\Delta='+str(Delta)+'$')
            plt.plot(xdata,ydata2,plotProps['markerType'],label=r'From rms speed')
        elif plotName=='virialTemperature':
            xdata=halos['mass']
            ydata=halos['temperature']
            ydata2=((u.km/u.s)**2*1e-27*u.kg/c.k_B).to(u.K)*halos['rms_speed']**2/3.
            # Relation expected for isothermal sphere for LCDM cosmology used in Sussing
            from astropy.cosmology import FlatLambdaCDM
            cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)
            ydata3=((10*c.M_sun*c.G*cosmo.H(0))**(2/3)*1e-27*u.kg/c.k_B).to(u.K)*(Delta/200)**(1/3)*0.5*xdata**(2/3)
            plt.plot(xdata,ydata3,'-',label='SIS model, $\Delta='+str(Delta)+'$')
            plt.plot(xdata,ydata2,plotProps['markerType'],label=r'From rms speed')
        elif plotName=='baryonFraction':
            xdata=halos['mass']
            ydata=halos['mass_baryon']/halos['mass']
            plt.plot(xdata,0.*xdata+baryonFraction,'-')
        elif plotName=='hotGas':
            xdata=halos['mass']
            ydata=halos['mass_gas_hot']
            plt.plot(xdata,xdata*baryonFraction,'-')
        elif plotName=='hotGasMetallicity':
            xdata=halos['mass']
            ydata=halos['mass_metals_gas_hot']/halos['mass_gas_hot']
        elif plotName=='ejectedGas':
            xdata=halos['mass']
            ydata=halos['mass_gas_eject']
            plt.plot(xdata,xdata*baryonFraction,'-')
        elif plotName=='ejectedGasMetallicity':
            xdata=halos['mass']
            ydata=halos['mass_metals_gas_eject']/halos['mass_gas_eject']
        elif plotName=='stellarHaloMassRatio':
            xdata=halos['mass']
            ydata=halos['mass_stars']/halos['mass']
            plt.ylim([0.,1.1*np.max(ydata)])
        elif plotName=='starFormationRate':
            xdata=halos['mass']
            ydata=halos['star_formation_rate']
        else:
            print('Plot type',plotName,'not yet implemented')
            continue # with next plot request
        plt.plot(xdata,ydata,plotProps['markerType'],label=plotProps['label'])
        #plt.plot([xdata[34],xdata[208]],[ydata[34],ydata[208]],'o')
        if plotProps['legend']: plt.legend()
        if plotProps['save_png']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.png',bbox_inches='tight')
        if plotProps['save_pdf']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.pdf',bbox_inches='tight')